# Importing the Dataframe

Load in dataframe made in previous notebook

In [221]:
import pandas as pd
df = pd.read_csv('data/sentiments (1).csv')
df.drop(columns = ["Unnamed: 0"], inplace = True) #takes out a column named unnamed: 0

In [222]:
df.head()

,positive,negative,neutral,num_positive,num_negative,num_neutral,filed_at
0,0.724790,0.648215,0.754609,52.0,26.0,45.0,2022-10-25T16:16:44-04:00
1,0.753925,0.684386,0.742942,44.0,28.0,48.0,2022-07-25T16:43:18-04:00
2,0.745917,0.672141,0.734098,36.0,17.0,47.0,2022-04-26T16:19:53-04:00
3,0.709038,0.671849,0.752585,42.0,45.0,46.0,2021-11-05T07:23:16-04:00
4,0.715977,0.673692,0.753832,48.0,37.0,44.0,2021-07-27T16:41:04-04:00


# Restructure the Dataframe for Better Analysis

The cell below creates variables that further narrow down the data to include the weighted average of each  "prediction" value (%).

This creates a "weighted" variable. This looks at the prediction variable in context of the 3 "num_***" variables to determine which label most accurately represents the true sentiment of a filing.

In [223]:
score = []
prediction = []
weighted = []
for i in enumerate(df.filed_at):
    pos_weighted = df.positive[i[0]]*df.num_positive[i[0]]/(df.num_positive[i[0]] + df.num_negative[i[0]] + df.num_neutral[i[0]])
    neg_weighted = df.negative[i[0]]*df.num_negative[i[0]]/(df.num_positive[i[0]] + df.num_negative[i[0]] + df.num_neutral[i[0]])
    neut_weighted = df.neutral[i[0]]*df.num_neutral[i[0]]/(df.num_positive[i[0]] + df.num_negative[i[0]] + df.num_neutral[i[0]])
    
    if pos_weighted > neg_weighted and pos_weighted > neut_weighted:
        score.append(df.positive[i[0]])
        prediction.append(0)
        weighted.append(pos_weighted)
        
    if neg_weighted > pos_weighted and neg_weighted > neut_weighted:
        score.append(df.negative[i[0]])
        prediction.append(1)
        weighted.append(neg_weighted)
        
    if neut_weighted > pos_weighted and neut_weighted > neg_weighted:
        
        if pos_weighted > neg_weighted:
            score.append(df.positive[i[0]])
            prediction.append(0)
            weighted.append(pos_weighted)
            
        if neg_weighted > pos_weighted:
            score.append(df.negative[i[0]])
            prediction.append(1)
            weighted.append(neg_weighted)
            
    if df.num_neutral[i[0]] == 0:
        score.append(0)
        prediction.append(0)
        weighted.append(0)
        

score[:4], prediction[:4], weighted[:4]

([0.7247903846153847, 0.753925, 0.7459166666666667, 0.6718488888888889],
 [0, 0, 0, 1],
 [0.30641544715447155, 0.2764391666666667, 0.26853, 0.22731729323308272])

The dataframe named "sentiments" is created and houses 4 variables as seen below. 

In [224]:
date = []
for i in df.filed_at:
     date.append(i[:10])

sentiments = pd.DataFrame()
sentiments["Date"] = date
sentiments["Date"] = pd.to_datetime(sentiments["Date"])
sentiments["score"] = score
sentiments["prediction"] = prediction
sentiments["weighted"] = weighted

# sentiments

In [225]:
sentiments.head()

,Date,score,prediction,weighted
0,2022-10-25,0.724790,0,0.306415
1,2022-07-25,0.753925,0,0.276439
2,2022-04-26,0.745917,0,0.268530
3,2021-11-05,0.671849,1,0.227317
4,2021-07-27,0.715977,0,0.266410


# Getting Stock Data:

This is not my code, this was written by a another for his project and he allowed me to use it. This section of code retrieves data related to IBM's stock price and other related metrics.

In [226]:
import requests
#import plotly.express as px #used for visuals
#from plotly.offline import init_notebook_mode, iplot
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt


#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected = True)

#import plotly

import datetime

# import warnings
# warnings.filterwarnings('ignore')
# from pylab import rcParams

In [227]:
import pandas as pd
import requests

def get_cash_flow_statement(ticker, limit, key, period):
    """Get the Cash flow statements."""
    URL = 'https://financialmodelingprep.com/api/v3/cash-flow-statement/'
    try:
        r = requests.get(
            '{}{}?period={}&?limit={}&apikey={}'.format(URL, ticker,period,limit, key))
        cashFlow = pd.DataFrame.from_dict(r.json()).transpose()
        cashFlow.columns = cashFlow.iloc[0]
        return cashFlow[1:]
    except requests.exceptions.HTTPError as e:
        print('Requesting Cash flow statement ERROR: ', str(e))


def get_financial_ratios(ticker, limit, key, period):
    """Period is ttm | annual | quarter."""
    URL = 'https://financialmodelingprep.com/api/v3/'
    if period == "ttm":
        try:
            r = requests.get(
                '{}/ratios-ttm/{}?{}&apikey={}'.format(URL,ticker,period, key))
            fr = pd.DataFrame.from_dict(r.json()).transpose()
            fr.columns = [ticker + " TTM Ratios"]
            return fr
        except requests.exceptions.HTTPError as e:
            print('Requesting Financial ratios ERROR(1): ', str(e))
    elif period == "annual" or period == "quarter":
        try:
            r = requests.get(
                '{}ratios/{}?period={}&?limit={}&apikey={}'.format(URL,ticker,period,limit,key))
            fr = pd.DataFrame.from_dict(r.json()).transpose()
            fr.columns = fr.iloc[1]
            return fr[2:]
        except requests.exceptions.HTTPError as e:
            print('Requesting Financial ratios ERROR(2): ', str(e))
    else:
        print('ERROR: Define the period you want: ttm | annual | quarter')
        return None


def get_key_metrics(ticker, limit, key, period):
    """Period is ttm | annual | quarter."""
    URL = 'https://financialmodelingprep.com/api/v3/'
    if period == "ttm":
        try:
            r = requests.get(
                '{}key-metrics-ttm/{}?apikey={}'.format(URL, ticker, key))
            km = pd.DataFrame.from_dict(r.json()).transpose()
            km.columns = [ticker + " TTM Ratios"]
            return km
        except requests.exceptions.HTTPError as e:
            print('Requesting Key Metrics ERROR(1): ', str(e))
    elif period == "annual" or period == "quarter":
        try:
            r = requests.get(
                '{}key-metrics/{}?period={}&?limit={}&apikey={}'.format(URL,ticker,period,limit,key))
            km = pd.DataFrame.from_dict(r.json()).transpose()
            km.columns = km.iloc[1]
            return km[2:]
        except requests.exceptions.HTTPError as e:
            print('Requesting Key Metrcs ERROR(2): ', str(e))
    else:
        print('ERROR: Define the period you want: ttm | annual | quarter')
        return None

In [228]:
key=pd.read_csv('data/key.txt',header=None)[0][0]
ticker =['IBM']
period='quarter'

In [229]:
ratios=get_financial_ratios(ticker='IBM',limit=40, key=key, period=period)
ratios=ratios.transpose()
ratios.dropna(axis=1)
ratios.shape

(115, 55)

In [230]:
metrics=get_key_metrics(ticker='IBM', limit=6, key=key, period='quarter')
metrics=metrics.transpose()
metrics=metrics.dropna(axis=1)
metrics.shape

(115, 56)

In [231]:
price=pd.read_csv('data/IBM.csv')
#price = price.drop(price.columns[[1,2,3,5]], axis=1)
price.shape

(7218, 7)

In [232]:
inner_join=pd.merge(ratios,metrics,on="date",how='inner')
inner_join.dropna()
inner_join=inner_join.reset_index(level=0)
inner_join.rename(columns={'date': 'Date'}, inplace=True)
inner_join.shape

(115, 112)

In [233]:
#data = data.reset_index(0) #make "date" a variable instead of an index
data=pd.merge(inner_join,price,on="Date",how='inner') #Combine Metrics&Ratios Data Set with Price

print(data.shape)
#data.drop(columns=data.columns[0:2], axis=1, inplace=True)
data=data.set_index('Date')
data=pd.get_dummies(data, columns=['period_x'])
# data.head()

(114, 118)


In [234]:
cols = data.columns
data[cols[1:]] = data[cols[1:]].apply(pd.to_numeric, errors='coerce')
data = data.apply(pd.to_numeric)
data.dtypes

currentRatio_x                float64
quickRatio                    float64
cashRatio                     float64
daysOfSalesOutstanding        float64
daysOfInventoryOutstanding    float64
                               ...   
Volume                          int64
period_x_Q1                     uint8
period_x_Q2                     uint8
period_x_Q3                     uint8
period_x_Q4                     uint8
Length: 120, dtype: object

In [235]:
data= data.drop('Open', axis = 1)
data= data.drop('High', axis = 1)
data= data.drop('Low', axis = 1)
data= data.drop('period_y', axis = 1)
data= data.drop('interestCoverage', axis = 1)
data= data.drop('Adj Close', axis = 1)

In [236]:
# Ensure we know the actual closing price
data1 = data[["Close"]]
data1 = data1.rename(columns = {'Close':'Actual_Close'})

# Setup our target.  This identifies if the price went up or down
data1["Movement"] = data.rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])["Close"]
data1 = data1.reset_index()
data1["Date"] = pd.to_datetime(data1["Date"])
data1 = data1[:74]
# data1 = data1.iloc[::-1]
# data1

# Testing the Accuracy of the Model

The following plots to help visualize the predicted vs actual movement of IBM's stock price

In [237]:
import plotly.express as px

print("The Actual Movement of Stock Price")
px.line(data1, x='Date', y="Movement").show() # actual

print("The Predicted Movement of Stock Price")
px.line(sentiments, x='Date', y="prediction").show() # predicted

The Actual Movement of Stock Price


The Predicted Movement of Stock Price


It's difficult to see if there is a relationship between the predicted and actual values. So, I'll try a different visualization

###  Change "data1.Date" to match "sentiments.Date"

Doing this will give you both:
- the "prediction": sentiment of a filing (positive or negative)
- the "movement": the directional movement of stock price from the current month to the nex (0 -> increase, 1 -> decrease)

In [238]:
data1

,Date,Actual_Close,Movement
0,2022-06-30,141.190002,NaN
1,2022-03-31,130.020004,0.0
2,2021-12-31,133.660004,1.0
3,2021-09-30,132.820267,0.0
4,2021-06-30,140.143402,1.0
...,...,...,...
69,2005-03-31,87.361374,1.0
70,2004-12-31,94.244743,1.0
71,2004-09-30,81.969406,0.0
72,2004-06-30,84.273422,1.0


In [239]:
data1["Date"] = data1.Date.dt.strftime('%m/%Y')
sentiments["Date"] = sentiments.Date.dt.strftime('%m/%Y')

### Change "data1.date" values to match up with the quarter its in

In [240]:
date = []
for i in enumerate(data1.Date):
    if data1.Date[i[0]][:2] == "03":
        date.append("03/"+ data1.Date[i[0]][3:7])
    if data1.Date[i[0]][:2] == "06":
        date.append("06/"+ data1.Date[i[0]][3:7])
    if data1.Date[i[0]][:2] == "09":
        date.append("09/"+ data1.Date[i[0]][3:7])
    if data1.Date[i[0]][:2] == "12":
        date.append("12/"+ data1.Date[i[0]][3:7])
# date

data1["date"] = date
data1.drop(columns = ["Date"], inplace = True)
data1



,Actual_Close,Movement,date
0,141.190002,NaN,06/2022
1,130.020004,0.0,03/2022
2,133.660004,1.0,12/2021
3,132.820267,0.0,09/2021
4,140.143402,1.0,06/2021
...,...,...,...
69,87.361374,1.0,03/2005
70,94.244743,1.0,12/2004
71,81.969406,0.0,09/2004
72,84.273422,1.0,06/2004


### Change the sentiment.date values to match the quarter its in

There were a few exceptions I had to account for, seen below.

In [241]:
date = []
for i in enumerate(sentiments.Date):
    # print(i[0])
    if i[1][:2] == "04" or i[1][:2] == "05":
        # print("01")
        date.append("03/" + sentiments.Date[i[0]][3:7])
    if i[1][:2] == "07":
        # print("02")
        date.append("06/" + sentiments.Date[i[0]][3:7])
    if i[1][:2] == "10" or i[1][:2] == "11":
        if i[0] == 0:
            # print("03")
            date.append("09/" + sentiments.Date[i[0]][3:7])
        else:
            if sentiments.Date[i[0]+1][:2] == "07":
                # print("03")
                date.append("09/" + sentiments.Date[i[0]][3:7])
            else:
                # print("04")
                date.append("12/" + sentiments.Date[i[0]][3:7])
# date

sentiments["date"] = date
sentiments.drop(columns = ["Date"], inplace = True)
sentiments[:4], sentiments[-4:]

(      score  prediction  weighted     date
 0  0.724790           0  0.306415  09/2022
 1  0.753925           0  0.276439  06/2022
 2  0.745917           0  0.268530  03/2022
 3  0.671849           1  0.227317  09/2021,
     score  prediction  weighted     date
 54    0.0           0       0.0  12/2004
 55    0.0           0       0.0  09/2004
 56    0.0           0       0.0  06/2004
 57    0.0           0       0.0  03/2004)

Took out the first 2 values in sentiments so that the it matches the date range in "data1", and fixed the indicies



In [242]:
sentiments = sentiments[2:]
sentiments = sentiments.reset_index()
sentiments = sentiments.drop(columns = ["index"])
sentiments.head()

,score,prediction,weighted,date
0,0.745917,0,0.268530,03/2022
1,0.671849,1,0.227317,09/2021
2,0.715977,0,0.266410,06/2021
3,0.716854,0,0.263748,03/2021
4,0.727618,0,0.250119,09/2020


### Match the 2 dataframes: data1 & sentiments, on their "date" variable

Make a temporary variable for the data1.date & sentiments.date along with the binary variable data1.Movement, to allow data1.Movement and sentiments.predictio to match up

In [243]:
data1 = data1[1:]
data1 = data1.reset_index()
data1 = data1.drop(columns = ["index"])
# data1[:]

dates1 = []

for i in enumerate(sentiments.date):
        dates1.append(i[1])


movement = []
dates2 = []
for i in enumerate(data1.date):
    if i[1] in dates1:
        movement.append(data1.Movement[i[0]])
        dates2.append(data1.date[i[0]])

# movement
dates1 == dates2

True

### Combine the 2 dataframes

Now, the model prediction can directly look at the actual increse and decrease of the model

In [244]:
sentiments["actual"] = movement

In [245]:
sentiments.head()

,score,prediction,weighted,date,actual
0,0.745917,0,0.268530,03/2022,0.0
1,0.671849,1,0.227317,09/2021,0.0
2,0.715977,0,0.266410,06/2021,1.0
3,0.716854,0,0.263748,03/2021,0.0
4,0.727618,0,0.250119,09/2020,0.0


### Change sentiments.date to pd.datetime

In [246]:
dates = []
for i in enumerate(sentiments.date):
    dates.append(i[1][-4:]+"/"+i[1][:3]+"01")
sentiments.date = pd.to_datetime(dates)

## Visualizations

In [247]:
import plotly.express as px

px.line(sentiments, x='date', y="prediction").show()

px.line(sentiments, x='date', y="actual").show()

From the above graphs it's difficult to discern if there is a relationship between the variables, so we move on to another visualization.

In [248]:
import plotly.express as px
# cm = confusionmatrix()
fig = px.density_heatmap(sentiments, x="prediction", y="actual", text_auto=True)
fig.show()

From this we see that the model's positive and negative sentiment score corresponds to the directional movement 28 out of 56 times.

Meaning, if you were to get a 10Q filing from IBM and ran it through the model I created in this notebook, there is a 50% chance that IBM's stock price will correspond to the positive and negative sentiment from the model.

## Conclusion

In this pair of notebooks, I show my process for APIs, data preprocessing, tokenization, word-embedding, and data visualization to show that there might be a correlation between the MDA section in 10Q filings and IBM's stock price.

In the end, with the data and model selected, I was not able to accurately use SEC filings as a predictor, however.

There is a good reason for that. The size of the data I used was only 57 data points. When going into this project, I had little knowledge of the financial terms and textual data analysis covered in this notebook. This project was used to teach myself by getting first hand experience navigating the relevant literature and replicating the processes that I see most.

### Future work

If I were to coninue this project I would:
- Incorporate more companies to get a better view of the predictive capabalities of this process. Rather than just this process's ability to predict IBM's stock price.
- Change up the way sentiments are generated to try and get a higher accuracy.
- Test different Natural Language Processing models to try and get more accurate sentiments.